# Algebra of Programming
- Dijkstra
- backhouse
- http://www.mathmeth.com/read.shtml
- Bird book
- AoP book
- Gries


https://arxiv.org/abs/2312.06103 equatonal reasoning dependent types. references gibbons paper
